In [1]:
import json
import numpy as np
import tensorflow as tf

In [2]:
n_batch = 10

## Load

In [3]:
with open('../json/computing_order.json', 'r') as f:
    computing_order = json.load(f)

with open('../json/formulas_light.json', 'r') as f:
    formulas_light = json.load(f)

with open('../json/constants_light.json', 'r') as f:
    constants_light = json.load(f)

with open('../json/inputs_light.json', 'r') as f:
    inputs_light = json.load(f)

with open('../json/unknowns_light.json', 'r') as f:
    unknowns_light = json.load(f)

with open('../json/input_variables.json', 'r') as f:
    input_variables = json.load(f)


## Index and reverse index for formulas, constants and input variables

In [4]:
n_formulas = len(computing_order)
reverse_index_formulas = computing_order
index_formulas = {
    v: i
    for i, v in enumerate(reverse_index_formulas)
}

In [5]:
n_inputs = len(inputs_light)
reverse_index_inputs = inputs_light
index_inputs = {
    v: i
    for i, v in enumerate(reverse_index_inputs)
}

## Create constants

In [6]:
tf_constant_zero = tf.constant(np.zeros(n_batch), dtype=tf.float64)
tf_constant_one = tf.constant(np.ones(n_batch), dtype=tf.float64)
tf_constant_false = tf.constant(np.zeros(n_batch, dtype=np.bool))
tf_constant_true = tf.constant(np.ones(n_batch, dtype=np.bool))

## Create input placeholder

In [7]:
tf_inputs = tf.placeholder(tf.float64, shape=(n_batch, n_inputs))

## Define functions

In [8]:
def produit(l):
    accu = tf_constant_one
    for i in range(len(l)):
        accu = tf.mul(accu, l[i])
    return accu

def dans(l):
    accu = tf_constant_false
    for i in range(1, len(l)):
        tmp = tf.equal(l[0], l[1])
        accu = tf.logical_or(accu, tmp)
    return tf.cast(accu, tf.float64)

def boolean_or(l):
    accu = tf_constant_false
    for i in range(len(l)):
        tmp = tf.cast(l[i], dtype=tf.bool)
        accu = tf.logical_or(accu, tmp)
    return tf.cast(accu, dtype=tf.float64)

def boolean_et(l):
    accu = tf_constant_true
    for i in range(len(l)):
        tmp = tf.cast(l[i], dtype=tf.bool)
        accu = tf.logical_and(accu, tmp)
    return tf.cast(accu, dtype=tf.float64)

def plus(l):
    return tf.add_n(l)

def moins(l):
    return tf.neg(l[0])

def positif(l):
    tmp = tf.greater(l[0], tf_constant_zero)
    return tf.cast(tmp, dtype=tf.float64)

def positif_ou_nul(l):
    tmp = tf.greater_equal(l[0], tf_constant_zero)
    return tf.cast(tmp, dtype=tf.float64)

def nul(l):
    tmp = tf.cast(l[0], dtype=tf.bool)
    tmp = tf.logical_not(tmp)
    return tf.cast(tmp, dtype=tf.float64)

def non_nul(l):
    tmp = tf.cast(l[0], dtype=tf.bool)
    return tf.cast(tmp, dtype=tf.float64)

def superieur_ou_egal(l):
    tmp = tf.greater_equal(l[0], l[1])
    return tf.cast(tmp, dtype=tf.float64)

def superieur(l):
    tmp = tf.greater(l[0], l[1])
    return tf.cast(tmp, dtype=tf.float64)

def inferieur(l):
    tmp = tf.greater(l[1], l[0])
    return tf.cast(tmp, dtype=tf.float64)

def egal(l):
    tmp = tf.equal(l[0], l[1])
    return tf.cast(tmp, dtype=tf.float64)

def ternaire(l):
    condition = tf.cast(l[0], dtype=tf.bool)
    return tf.select(condition, l[1], l[2])

def si(l):
    condition = tf.cast(l[0], dtype=tf.bool)
    return tf.select(condition, l[1], tf_constant_zero)

def inverse(l):
    inv = tf.inv(l[0])
    est_non_nul = tf.cast(l[0], dtype=bool)
    return tf.select(est_non_nul, inv, tf_constant_zero)

def maximum(l):
    accu = l[0]
    for i in range(1, len(l)):
        accu = tf.maximum(accu, l[i])
    return accu

def minimum(l):
    accu = l[0]
    for i in range(1, len(l)):
        accu = tf.minimum(accu, l[i])
    return accu

def plancher(l):
    return tf.floor(l[0])

def arrondi(l):
    return tf.round(l[0])

def absolue(l):
    return np.abs(l[0])


functions_mapping = {
    '+': plus,
    '*': produit,
    '-': moins,
    'unary:-': moins,
    'positif': positif,
    'positif_ou_nul': positif_ou_nul,
    'null': nul,
    'operator:>=': superieur_ou_egal,
    'operator:>': superieur,
    'operator:<': inferieur,
    'operator:=': egal,
    'ternary': ternaire,
    'si': si,
    'inverse': inverse,
    'max': maximum,
    'min': minimum,
    'inf': plancher,
    'arr': arrondi,
    'abs': absolue,
    'present': non_nul,
    'boolean:ou': boolean_or,
    'boolean:et': boolean_et,
    'dans': dans
}


## Build graph

In [9]:
var = computing_order[1]

In [10]:
formulas_light[var]

{'args': [{'name': 'V_0CF', 'nodetype': 'symbol'},
  {'name': 'V_0DJ', 'nodetype': 'symbol'}],
 'name': '+',
 'nodetype': 'call'}

In [11]:
def build_graph(node):
    if node['nodetype'] == 'float':
        scalar_constant = tf.constant(node['value'], dtype=tf.float64)
        vector_constant = tf.mul(tf_constant_one, scalar_constant)
        return vector_constant
    
    if node['nodetype'] == 'symbol':
        name = node['name']
        if name in formulas_light:
            return tf_formulas[name]

        if name in constants_light:
            value = constants_light[name]
            return build_graph({'nodetype': 'float', 'value': value})

        if name in inputs_light:
            index = index_inputs[name]
            begin = [0, index]
            size = [n_batch, 1]
            tmp = tf.slice(tf_inputs, begin, size)
            return tf.reshape(tmp, [n_batch])

        if name in unknowns_light:
            return tf_constant_zero

        raise Exception('Unknown variable category.')

    if node['nodetype'] == 'call':
        name = node['name']
        args = [build_graph(child) for child in node['args']]
        function = functions_mapping[name]
        value = function(args)
        return value

    raise ValueError('Unknown type : %s'%nodetype)


In [12]:
tf_formulas = {}
for var in computing_order:
    tf_formulas[var] = build_graph(formulas_light[var])

## Fetch (or generate) inputs

In [13]:
alias2name = {i['alias']: i['name'] for i in input_variables}

alias = '1AJ'
name = alias2name[alias]
index = index_inputs[name]

In [14]:
inputs = np.zeros((n_batch, n_inputs))
inputs[:, index] = 30000.

## Make the computation

In [15]:
with tf.Session() as sess:
    result = sess.run(tf_formulas['IRN'], feed_dict={tf_inputs: inputs})